Implementation of an AVL Tree with duplicate keys

In [4]:
class Node(object):
    def __init__(self, key, id_):
        self.value = key
        self.ids = [id_]  # list of IDs for duplicate keys
        self.left = None
        self.right = None
        self.height = 1

class AVLTree(object):

    def getHeight(self, root):
        if root is None:
            return 0
        return root.height

    def getBal(self, root):
        if root is None:
            return 0
        return self.getHeight(root.left) - self.getHeight(root.right)

    def leftRotate(self, r):
        y = r.right
        z = y.left
        y.left = r
        r.right = z
        r.height = 1 + max(self.getHeight(r.left), self.getHeight(r.right))
        y.height = 1 + max(self.getHeight(y.left), self.getHeight(y.right))
        return y

    def rightRotate(self, r):
        y = r.left
        z = y.right
        y.right = r
        r.left = z
        r.height = 1 + max(self.getHeight(r.left), self.getHeight(r.right))
        y.height = 1 + max(self.getHeight(y.left), self.getHeight(y.right))
        return y

    def insert(self, root, key, id_): 
        # standard BST insertion
        if root is None:
            return Node(key, id_)
        elif key < root.value:
            root.left = self.insert(root.left, key, id_)
        elif key > root.value:
            root.right = self.insert(root.right, key, id_)
        else:
            root.ids.append(id_)
            return root

        # Update the height along the path
        root.height = 1 + max(self.getHeight(root.left), self.getHeight(root.right))

        # Check balance and restore balance as needed
        b = self.getBal(root)

        if b == 2:  # The left subtree is taller and out of balance
            if key < root.left.value:
                # key is inserted in the left subtree rooted at root.left.left, off balance
                return self.rightRotate(root)
            elif key > root.left.value:
                # key is inserted in the right subtree rooted at root.left.right, off balance
                root.left = self.leftRotate(root.left)
                return self.rightRotate(root)

        if b == -2:  # The right subtree is taller and out of balance
            if key > root.right.value:
                # key is inserted in the right subtree rooted at root.right.right, off balance
                return self.leftRotate(root)
            elif key < root.right.value:
                # key is inserted in the left subtree rooted at root.right.left, off balance
                root.right = self.rightRotate(root.right)
                return self.leftRotate(root)

        return root

    def inOrder(self, root):  # inorder traversal sorts the keys in increasing order
        if not root:
            return
        self.inOrder(root.left)
        print(f"Key: {root.value}, IDs: {root.ids}")
        self.inOrder(root.right)

    def getMinValueNode(self, root):
        if root is None or root.left is None:
            return root  # the value of the root is the smallest
        return self.getMinValueNode(root.left)  # keep moving down along the left nodes until Null
    

    def search(self, root, x):  # return the pointer that points to the node found
        if root is None:
            print(x, "is not found")
            return root
        elif x == root.value:
            print(x, "is found")
            return root
        elif x < root.value:
            self.search(root.left, x)
        else:
            self.search(root.right, x)

    # Recursive function to delete a node with the given key from subtree with given root.
    # returns root of the modified subtree.
    def delete(self, root, key, id_=None):

        # Perform the standard BST delete
        if root is None:
            print(key, "is not found.")
            return root
        elif key < root.value:
            root.left = self.delete(root.left, key, id_)
        elif key > root.value:
            root.right = self.delete(root.right, key, id_)
        else:
            if id_ is not None and id_ in root.ids:
                root.ids.remove(id_)
                if len(root.ids) > 0:
                    return root

            if root.left is None:
                return root.right
            elif root.right is None:
                return root.left

            temp = self.getMinValueNode(root.right)
            root.value = temp.value
            root.ids = temp.ids.copy()
            root.right = self.delete(root.right, temp.value)

        # Update the height of the ancestor node
        root.height = 1 + max(self.getHeight(root.left), self.getHeight(root.right))

        # Check balance and restore balance when needed
        b = self.getBal(root)

        # The left subtree is taller and off balance
        if b == 2:
            if self.getBal(root.left) >= 0:
                return self.rightRotate(root)  # Case 1 - Left Left
            elif self.getBal(root.left) < 0:  # Case 2 - Left Right
                root.left = self.leftRotate(root.left)
                return self.rightRotate(root)

        # The right subtree is taller and off balance
        if b == -2:
            if self.getBal(root.right) <= 0:
                return self.leftRotate(root)  # Case 3 - Right Right
            elif self.getBal(root.right) > 0:  # Case 4 - Right Left
                root.right = self.rightRotate(root.right)
                return self.leftRotate(root)

        # Returns root
        return root
    

In [10]:
Tree = AVLTree()
root = None

# Insert nodes
print(f'Base AVL with duplicate keys')
nodes = [(5, 'a'), (3, 'b'), (3, 'c'), (7, 'd'), (5, 'e')]
for key, id_ in nodes:
    root = Tree.insert(root, key, id_)

Tree.inOrder(root)
print()

# Delete specific IDs and observe the structure
print(f'Removed specific key')
root = Tree.delete(root, 3, 'b')
Tree.inOrder(root)
print()

print(f'Removed another key')
root = Tree.delete(root, 5, 'a')
Tree.inOrder(root)
print()


Base AVL with duplicate keys
Key: 3, IDs: ['b', 'c']
Key: 5, IDs: ['a', 'e']
Key: 7, IDs: ['d']

Removed specific key
Key: 3, IDs: ['c']
Key: 5, IDs: ['a', 'e']
Key: 7, IDs: ['d']

Removed another key
Key: 3, IDs: ['c']
Key: 5, IDs: ['e']
Key: 7, IDs: ['d']

